# Uebung 2

**Aufgabe 6**

Betrachten Sie das einfache lineare Regressionsmodell
$$M : \text{Defekt}_i = β_0 + β_1 \text{Alter} _i + ϵ_i, i = 1, . . . , 13$$

Für glaukom.txt

**A:** Berechnen Sie die kleinste Quadrate Schätzer $\hat{β_0}$ und $\hat{β_1}$ und die Varianz-Kovarianz Matrix für die kleinsten Quadrate Schätzer $\hat{β_0}$ und $\hat{β_1}$.

In [12]:
import pandas as pd
import numpy  as np
from sklearn.linear_model import LinearRegression

glaukom = pd.read_csv('Glaukom.txt', sep='\t')

print(glaukom)

   lnr  Defekt  Flimmertest  Alter  Geschlecht
0    1     6.1         13.2     46           0
1    2     9.7         12.5     51           1
2    3     4.1         15.5     25           1
3    4     8.8         10.1     59           0
4    5     4.0         12.5     23           1
5    6     7.9         11.9     54           0
6    7     7.4          8.9     44           0
7    8     6.4         10.2     42           0
8    9    10.4          5.4     47           1
9   10    10.3          8.3     50           1


In [24]:
defect, age = np.array(glaukom['Defekt']), np.array(glaukom['Alter'])

# X = age transponiert
X = age.reshape(-1, 1)
y = defect.reshape(-1, 1)

model = LinearRegression()
model.fit(X, y)

beta_0 = model.intercept_[0]
beta_1 = model.coef_[0][0]

print(f"Koeffizienten:")
print(f"Intercept Beta_0 = {beta_0}")
print(f"Slope Beta_1 = {beta_1}")

Koeffizienten:
Intercept Beta_0 = 0.26502563267963275
Slope Beta_1 = 0.16428513304581333


Die Varianz-Kovarianz-Matrix für $\hat{β_0}$ und $\hat{β_1}$ wird wie folgt berechnet:

$$
\hat{Cov(\hat{β})} = \sigma ^2 (X^T X)^{-1}
$$

Wobei $\sigma ^2$ die Varianz unserer $\epsilon$ ist

In [56]:
y_pred = model.predict(X)

# Unsere "Residuals" sind unsere Epsilon, also die Abweichung unseren Modells von dem tatsächlichen Datenpunkt
residuals = y - y_pred

sigma_squared = np.var(residuals)

cov_beta_hat = sigma_squared * np.linalg.inv(X.T @ X)

print(cov_beta_hat)

[[8.10633771e-05]]


**B:** Sei
$$
\hat{y} = \hat{β_0} + \hat{β_0}x, x ∈ \mathbb{R}
$$

die Regressionsgerade für das Modell M. Berechnen Sie für jedes $x ∈ \mathbb{R}$
die Varianz des zugehörigen $\hat{y}$.